In [57]:
import itertools
import numpy as np
import json
from collections import defaultdict
from hog import *
import tensorflow as tf

In [11]:
training_data_file = 'mpii_data/train.json'

with open(training_data_file, 'r') as file:
    training_data = json.load(file)


In [13]:
print(training_data[0].keys())

dict_keys(['joints_vis', 'joints', 'image', 'scale', 'center'])


In [19]:
# Map images to data
image_to_data = defaultdict(list)
for data in training_data:
    image_to_data[data["image"]].append(data)

In [26]:
# Filter for single person images
single_person_images = {key: data[0] for key, data in image_to_data.items() if len(data) == 1}

In [32]:
def filter_for_single_person_data(data_filepath, save_to_filepath):
    with open(data_filepath, 'r') as file:
        loaded_data = json.load(file)
    
    image_to_data = defaultdict(list)
    for data in loaded_data:
        image_to_data[data["image"]].append(data)

    single_person_images = {key: data[0] for key, data in image_to_data.items() if len(data) == 1}

    single_person_json = [data for data in loaded_data if data["image"] in single_person_images.keys()]

    with open(save_to_filepath, 'w') as write_to:
        json.dump(single_person_json, write_to)
    print(f"Filtered data saved to {save_to_filepath}")

    return single_person_images

In [33]:
data_jsons = ["test.json", "train.json", "trainval.json", "valid.json"]
filtered_data_dicts = {}
for data_json_file in data_jsons:
    save_to = "filtered_data/single_person_" + data_json_file
    filtered_data_dicts[data_json_file] = filter_for_single_person_data("mpii_data/" + data_json_file, save_to)

Filtered data saved to filtered_data/single_person_test.json
Filtered data saved to filtered_data/single_person_train.json
Filtered data saved to filtered_data/single_person_trainval.json
Filtered data saved to filtered_data/single_person_valid.json


In [37]:
with open("filtered_data/single_person_train.json", 'r') as file:
    loaded_data = json.load(file)

image_to_data = defaultdict(list)
for data in loaded_data:
    image_to_data[data["image"]].append(data)

single_person_images = {key: data[0] for key, data in image_to_data.items() if len(data) == 1}

single_person_json = [data for data in loaded_data if data["image"] in single_person_images.keys()]

short = single_person_json[:100]


with open("filtered_data/single_person_small_train.json", 'w') as write_to:
    json.dump(short, write_to)


In [63]:
def preprocess_data(data_file, name):
    X = []
    y = []

    # Load training dataset
    with open(data_file, "r") as read_file:
        train_images = json.load(read_file)
    for image_obj in train_images:
        X.append(hog_feature("images/" + image_obj["image"]))
        y.append(np.ravel(image_obj["joints"]))
    X = np.column_stack((itertools.zip_longest(*X, fillvalue=0)))
    y = np.asarray(y)

    np.save(f"tf_data/{name}_X.npy", X)
    np.save(f"tf_data/{name}_y.npy", y)

    return X, y

In [65]:
X, y = preprocess_data("filtered_data/single_person_small_train.json", "small_train", )

In [64]:
X_train, y_train = preprocess_data("filtered_data/single_person_train.json", "train")


In [ ]:
X_test, y_test = preprocess_data("filtered_data/single_person_test.json", "test")

In [ ]:
X_val, y_val = preprocess_data("filtered_data/single_person_valid.json", "valid")